In [1]:
!pip install keras-tuner -q

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
def build_model(hp):
  units = hp.Int(name="units", min_value=16, max_value=64, step=16)
  model = keras.Sequential([
      layers.Dense(units, activation='relu'),
      layers.Dense(10, activation='softmax')
  ])
  optimizer = hp.Choice(name='optimizer', values=['rmsprop', 'adam'])
  model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [4]:
#Creating a KerasTuner Hyper Model

import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
  def __init__(self, num_classes):
    self.num_classes = num_classes

  def build(self, hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation='relu'),
        layers.Dense(self.num_classes, activation='softmax')
    ])
    optimizer = hp.choice(name='optimizer', values=['rmsprop', 'adam'])
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
hypermodel = SimpleMLP(10)

In [6]:
tuner = kt.BayesianOptimization(build_model, 
                                objective='val_accuracy',
                                max_trials=25,
                                executions_per_trial=2,
                                directory='mnist_kl_dataset',
                                overwrite=True)

In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [8]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:] 
y_train_full = y_train[:] 

In [9]:
num_val_samples = 10000 

x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:] 
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:] 

callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)]
tuner.search(x_train, y_train,
             batch_size=128,
             epochs=25, 
             validation_data=(x_val, y_val),
             callbacks=callbacks,
            verbose=2)

Trial 100 Complete [00h 00m 45s]
val_accuracy: 0.9740999937057495

Best val_accuracy So Far: 0.9760499894618988
Total elapsed time: 01h 45m 25s
INFO:tensorflow:Oracle triggered exit


In [12]:
best_hps = tuner.get_best_hyperparameters(4)
best_hps

In [13]:
def get_best_epoch(hp):
  model = build_model(hp)
  callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10)]
  history = model.fit(x_train, y_train,
                      validation_data=(x_val, y_val),
                      epochs=25,
                      batch_size=64,
                      callbacks=callbacks)
  
  val_loss_per_epoch = history.history['val_loss']
  best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
  print(f"Best epoch: {best_epoch}")
  return best_epoch

In [15]:
def get_best_trained_model(hp):
  best_epoch = get_best_epoch(hp)
  model = build_model(hp)
  model.fit(x_train_full, y_train_full, batch_size=64, epochs=(int(best_epoch*1.2)))
  return model

best_models = []
for hp in best_hps:
 model = get_best_trained_model(hp)
 model.evaluate(x_test, y_test)
 best_models.append(model)

Epoch 1/25
782/782 [==============================] - 3s 3ms/step - loss: 0.3941 - accuracy: 0.8908 - val_loss: 0.2131 - val_accuracy: 0.9406
Epoch 2/25
782/782 [==============================] - 2s 3ms/step - loss: 0.1995 - accuracy: 0.9422 - val_loss: 0.1687 - val_accuracy: 0.9522
Epoch 3/25
782/782 [==============================] - 2s 3ms/step - loss: 0.1504 - accuracy: 0.9565 - val_loss: 0.1352 - val_accuracy: 0.9622
Epoch 4/25
782/782 [==============================] - 2s 3ms/step - loss: 0.1209 - accuracy: 0.9649 - val_loss: 0.1248 - val_accuracy: 0.9630
Epoch 5/25
782/782 [==============================] - 2s 3ms/step - loss: 0.1025 - accuracy: 0.9706 - val_loss: 0.1151 - val_accuracy: 0.9659
Epoch 6/25
782/782 [==============================] - 2s 3ms/step - loss: 0.0867 - accuracy: 0.9742 - val_loss: 0.1084 - val_accuracy: 0.9680
Epoch 7/25
782/782 [==============================] - 2s 3ms/step - loss: 0.0752 - accuracy: 0.9779 - val_loss: 0.1002 - val_accuracy: 0.9711
Epoch 